In [28]:
import pandas as pd

In [29]:
prediction_table_prepath = '../data/prediction/'
df_recom_path = '../data/db_dump/20210805_get_recom_qty_dump.csv'
df_sales_path = '../data/db_dump/20210805_get_venta_neta_dump.csv'

In [30]:
df_recom = pd.read_csv(df_recom_path)
df_recom.recomm_qt.fillna(0, inplace=True)
df_recom['recomm_qt'] = df_recom['recomm_qt'].astype(int)
df_recom['fecha_de_visita'] = df_recom.fecha_de_visita.astype(str)
df_recom['codigo_de_cliente'] = df_recom.codigo_de_cliente.astype(str)

df_sales = pd.read_csv(df_sales_path)

df_sales.fecha_de_visita = df_sales.fecha_de_visita.astype(str)
df_sales.codigo_de_cliente = df_sales.codigo_de_cliente.astype(str)
df_sales.codigo_de_producto = df_sales.codigo_de_producto.astype(str)

In [31]:
prediction_table = pd.read_csv(prediction_table_prepath+'lgb_without_optuna_heldout_format.csv')

In [32]:
prediction_table.head()

,codigo_de_cliente,fecha_de_visita,codigo_de_producto,prediction probability,bought_in_the_visit,rank
0,94225070,2021-05-31,500040,0.409008,1.0,1
1,94225070,2021-05-31,500574,0.327965,0.0,2
2,94225070,2021-05-31,508588,0.314524,1.0,3
3,94225070,2021-05-31,500176,0.299598,1.0,4
4,94225070,2021-05-31,509625,0.298181,0.0,5


In [33]:
prediction_table['fecha_de_visita'] = prediction_table.fecha_de_visita.astype(str)
prediction_table['codigo_de_cliente'] = prediction_table.codigo_de_cliente.astype(str)
prediction_table['codigo_de_producto'] = prediction_table.codigo_de_producto.astype(str)

In [34]:
date = '2021-04-10'

prediction_table = prediction_table[prediction_table['fecha_de_visita'] == date]

prediction_table = prediction_table[(prediction_table['codigo_de_cliente']=='94161830') | 
                                   (prediction_table['codigo_de_cliente']=='9460242') | 
                                   (prediction_table['codigo_de_cliente']=='9426428')]

In [35]:



prediction_table = pd.merge(prediction_table, df_recom, how='left', left_on=['codigo_de_cliente', 'fecha_de_visita'], right_on=['codigo_de_cliente', 'fecha_de_visita'])


prediction_table_temp = prediction_table[prediction_table['rank'] > prediction_table['recomm_qt']]

prediction_table = prediction_table[prediction_table['rank'] <= prediction_table['recomm_qt']]



prediction_table = pd.merge(prediction_table, df_sales, how='left', left_on=['codigo_de_cliente', 'fecha_de_visita', 'codigo_de_producto'], right_on=['codigo_de_cliente', 'fecha_de_visita', 'codigo_de_producto'])

prediction_table['venta_neta_dolar'] = prediction_table['venta_neta_dolar'].fillna(0.00)
prediction_table['executed_flag'] = prediction_table['venta_neta_dolar'].apply(lambda x: 1 if x > 0.00 else 0)


# client_level_precision = prediction_table.groupby(['codigo_de_cliente']).agg({'codigo_de_producto': 'count', 'executed_flag': 'sum'}).reset_index()
# client_level_precision['precision'] = client_level_precision['executed_flag']/client_level_precision['codigo_de_producto']

# client_and_visit_level_precision = prediction_table.groupby(['fecha_de_visita', 'codigo_de_cliente']).agg({'codigo_de_producto': 'count', 'executed_flag': 'sum'}).reset_index()
# client_and_visit_level_precision['precision'] = client_and_visit_level_precision['executed_flag']/client_and_visit_level_precision['codigo_de_producto']

# precision = client_and_visit_level_precision['precision'].mean()

In [36]:
prediction_table.shape, prediction_table_temp.shape

((5, 9), (7, 7))

In [37]:
prediction_table_temp.to_csv('clients_pred_non_cutted_small_clients.csv', index=False)

In [38]:
prediction_table_temp

,codigo_de_cliente,fecha_de_visita,codigo_de_producto,prediction probability,bought_in_the_visit,rank,recomm_qt
2,94161830,2021-04-10,509506,0.509004,0.0,3,2
3,94161830,2021-04-10,508588,0.356748,1.0,4,2
6,9460242,2021-04-10,500574,0.247667,1.0,3,2
7,9460242,2021-04-10,508588,0.192120,0.0,4,2
9,9426428,2021-04-10,500366,0.503856,0.0,2,1
10,9426428,2021-04-10,500574,0.407497,0.0,3,1
11,9426428,2021-04-10,508403,0.266802,0.0,4,1


In [39]:
prediction_table

,codigo_de_cliente,fecha_de_visita,codigo_de_producto,prediction probability,bought_in_the_visit,rank,recomm_qt,venta_neta_dolar,executed_flag
0,94161830,2021-04-10,509603,0.639261,0.0,1,2,0.00,0
1,94161830,2021-04-10,510701,0.625517,0.0,2,2,0.00,0
2,9460242,2021-04-10,500040,0.826536,1.0,1,2,8417.34,1
3,9460242,2021-04-10,508720,0.523676,0.0,2,2,0.00,0
4,9426428,2021-04-10,500040,0.583397,1.0,1,1,10100.00,1


In [40]:
prediction_table

,codigo_de_cliente,fecha_de_visita,codigo_de_producto,prediction probability,bought_in_the_visit,rank,recomm_qt,venta_neta_dolar,executed_flag
0,94161830,2021-04-10,509603,0.639261,0.0,1,2,0.00,0
1,94161830,2021-04-10,510701,0.625517,0.0,2,2,0.00,0
2,9460242,2021-04-10,500040,0.826536,1.0,1,2,8417.34,1
3,9460242,2021-04-10,508720,0.523676,0.0,2,2,0.00,0
4,9426428,2021-04-10,500040,0.583397,1.0,1,1,10100.00,1


In [41]:
prediction_table.to_csv('clients_pred_small_clients.csv', index=False)